In [19]:
from PIL import Image
import os

def split_image(image_path, output_folder):
    # 打开图片
    img = Image.open(image_path)
    img_width, img_height = img.size

    # 计算每张小图的宽度和高度
    tile_width = img_width // 2
    tile_height = img_height // 2

    # 切割图片并保存
    for i in range(2):
        for j in range(2):
            left = j * tile_width
            upper = i * tile_height
            right = left + tile_width
            lower = upper + tile_height

            # 切割图片
            tile = img.crop((left, upper, right, lower))

            # 保存切割后的小图
            tile_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_tile_{i}_{j}.png"
            tile.save(os.path.join(output_folder, tile_name))

            # 对小图进行旋转
            for angle in [90, 180, 270]:
                rotated_tile = tile.rotate(angle, expand=True)
                rotated_tile_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_tile_{i}_{j}_rotated_{angle}.png"
                rotated_tile.save(os.path.join(output_folder, rotated_tile_name))

def process_folder(input_folder, output_folder):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历输入文件夹中的所有图片
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(input_folder, filename)
            split_image(image_path, output_folder)

if __name__ == "__main__":
    input_folder = r"L:\常惠林\萎凋\自然萎凋\原始\3萎凋过度"
    output_folder = r"L:\常惠林\萎凋\自然萎凋\原始\class_3"
    process_folder(input_folder, output_folder)

In [5]:
from PIL import Image
import numpy as np
import os

# 设置图片所在的目录路径
folder_path = r'L:\常惠林\萎凋\自然萎凋\分类\class_3'
output_folder = r'L:\常惠林\萎凋\自然萎凋\分类\class_3_四通道'

# 创建输出文件夹
os.makedirs(output_folder, exist_ok=True)

# 获取所有图片文件
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

# 对每个图像进行处理
for image_file in image_files:
    # 构建图像文件路径
    img_path = os.path.join(folder_path, image_file)
    
    # 读取图像（彩色图像）
    img = Image.open(img_path)
    
    # 转换为灰度图像
    img_gray = img.convert('L')  # 'L' 模式表示灰度图像
    
    # 将图像和灰度图像转化为 NumPy 数组
    img_array = np.array(img)  # RGB 图像
    img_gray_array = np.array(img_gray)  # 灰度图像
    
    # 将灰度图像扩展为三个通道（复制灰度图像到三个通道）
    img_gray_array = np.stack([img_gray_array] * 3, axis=-1)
    
    # 合并原始 RGB 图像和灰度图像（灰度图像作为额外的通道）
    combined_image = np.concatenate((img_array, img_gray_array[:, :, :1]), axis=-1)  # 添加灰度作为第四通道
    
    # 保存处理后的图像
    output_path = os.path.join(output_folder, f"combined_{image_file}")
    Image.fromarray(combined_image.astype(np.uint8)).save(output_path)
    print(f"已保存合并图像：{image_file} -> combined_{image_file}")

print("图像保存完成！")


已保存合并图像：104_tile_0_0.png -> combined_104_tile_0_0.png
已保存合并图像：104_tile_0_0_rotated_180.png -> combined_104_tile_0_0_rotated_180.png
已保存合并图像：104_tile_0_0_rotated_270.png -> combined_104_tile_0_0_rotated_270.png
已保存合并图像：104_tile_0_0_rotated_90.png -> combined_104_tile_0_0_rotated_90.png
已保存合并图像：104_tile_0_1.png -> combined_104_tile_0_1.png
已保存合并图像：104_tile_0_1_rotated_180.png -> combined_104_tile_0_1_rotated_180.png
已保存合并图像：104_tile_0_1_rotated_270.png -> combined_104_tile_0_1_rotated_270.png
已保存合并图像：104_tile_0_1_rotated_90.png -> combined_104_tile_0_1_rotated_90.png
已保存合并图像：104_tile_0_2.png -> combined_104_tile_0_2.png
已保存合并图像：104_tile_0_2_rotated_180.png -> combined_104_tile_0_2_rotated_180.png
已保存合并图像：104_tile_0_2_rotated_270.png -> combined_104_tile_0_2_rotated_270.png
已保存合并图像：104_tile_0_2_rotated_90.png -> combined_104_tile_0_2_rotated_90.png
已保存合并图像：104_tile_1_0.png -> combined_104_tile_1_0.png
已保存合并图像：104_tile_1_0_rotated_180.png -> combined_104_tile_1_0_rotated_180.png
已保存合并图像：10

In [6]:
from PIL import Image

# 读取图像
img_path = "L:\常惠林\萎凋\自然萎凋\四通道\class_1\combined_img_001.png"  # 替换为实际图像路径
image = Image.open(img_path)

# 获取图像的模式和通道数
mode = image.mode
if mode == 'RGB':
    print("该图像有3个通道（RGB）。")
elif mode == 'RGBA':
    print("该图像有4个通道（RGBA）。")
elif mode == 'L':
    print("该图像有1个通道（灰度）。")
else:
    print(f"该图像的模式为 {mode}，通道数未知。")


该图像有4个通道（RGBA）。


In [8]:
import os
import random
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset

# 定义添加高斯噪声的函数
def add_gaussian_noise(image_tensor, mean=0.0, std=0.1):
    # 生成高斯噪声
    noise = torch.randn_like(image_tensor) * std + mean
    
    # 添加噪声到图像
    noisy_image = image_tensor + noise
    
    # Clip values to make sure they are within the valid range [0, 1]
    noisy_image = torch.clamp(noisy_image, 0, 1)
    
    return noisy_image

# 定义数据扩增的转换操作
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomVerticalFlip(),    # 随机垂直翻转
    transforms.RandomRotation(30),      # 随机旋转30度
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 随机裁剪并缩放到224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 改变亮度、对比度、饱和度和色调
    transforms.ToTensor(),              # 转换为Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 图像归一化
])

# 定义数据集类
class AugmentedDataset(Dataset):
    def __init__(self, root_dir, transform=None, add_noise=True):
        self.root_dir = root_dir
        self.transform = transform
        self.add_noise = add_noise
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')  # 确保图像是RGB格式

        # 应用数据扩增
        if self.transform:
            image = self.transform(image)
        
        # 添加噪声
        if self.add_noise:
            image = add_gaussian_noise(image)

        return image

# 加载数据集
dataset = AugmentedDataset(root_dir=r"L:\常惠林\萎凋\自然萎凋\分类\class_1", transform=transform)

# 生成扩增后的图像并保存
output_folder = "augmented_with_noise_images"  # 设置保存扩增图像的文件夹
os.makedirs(output_folder, exist_ok=True)

for idx in range(len(dataset)):
    image = dataset[idx]
    img_name = f"augmented_with_noise_{idx}.png"
    img_path = os.path.join(output_folder, img_name)
    
    # 转换为PIL图像以便保存
    image_pil = transforms.ToPILImage()(image)
    image_pil.save(img_path)
    print(f"保存扩增并添加噪声的图像: {img_name}")
print("图像保存完成！")

保存扩增并添加噪声的图像: augmented_with_noise_0.png
保存扩增并添加噪声的图像: augmented_with_noise_1.png
保存扩增并添加噪声的图像: augmented_with_noise_2.png
保存扩增并添加噪声的图像: augmented_with_noise_3.png
保存扩增并添加噪声的图像: augmented_with_noise_4.png
保存扩增并添加噪声的图像: augmented_with_noise_5.png
保存扩增并添加噪声的图像: augmented_with_noise_6.png
保存扩增并添加噪声的图像: augmented_with_noise_7.png
保存扩增并添加噪声的图像: augmented_with_noise_8.png
保存扩增并添加噪声的图像: augmented_with_noise_9.png
保存扩增并添加噪声的图像: augmented_with_noise_10.png
保存扩增并添加噪声的图像: augmented_with_noise_11.png
保存扩增并添加噪声的图像: augmented_with_noise_12.png
保存扩增并添加噪声的图像: augmented_with_noise_13.png
保存扩增并添加噪声的图像: augmented_with_noise_14.png
保存扩增并添加噪声的图像: augmented_with_noise_15.png
保存扩增并添加噪声的图像: augmented_with_noise_16.png
保存扩增并添加噪声的图像: augmented_with_noise_17.png
保存扩增并添加噪声的图像: augmented_with_noise_18.png
保存扩增并添加噪声的图像: augmented_with_noise_19.png
保存扩增并添加噪声的图像: augmented_with_noise_20.png
保存扩增并添加噪声的图像: augmented_with_noise_21.png
保存扩增并添加噪声的图像: augmented_with_noise_22.png
保存扩增并添加噪声的图像: augmented_with_noise_23.png
保存

In [12]:
import os
import random
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset

# 设置设备：如果有GPU，就使用GPU，否则使用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义添加高斯噪声的函数
def add_gaussian_noise(image_tensor, mean=0.0, std=0.1):
    # 生成高斯噪声
    noise = torch.randn_like(image_tensor) * std + mean
    
    # 添加噪声到图像
    noisy_image = image_tensor + noise
    
    # Clip values to make sure they are within the valid range [0, 1]
    noisy_image = torch.clamp(noisy_image, 0, 1)
    
    return noisy_image

# 定义数据扩增的转换操作
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomVerticalFlip(),    # 随机垂直翻转
    transforms.RandomRotation(30),      # 随机旋转30度
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 随机裁剪并缩放到224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 改变亮度、对比度、饱和度和色调
    transforms.ToTensor(),              # 转换为Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 图像归一化
])

# 定义数据集类
class AugmentedDataset(Dataset):
    def __init__(self, root_dir, transform=None, add_noise=True):
        self.root_dir = root_dir
        self.transform = transform
        self.add_noise = add_noise
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')  # 确保图像是RGB格式

        # 应用数据扩增
        if self.transform:
            image = self.transform(image)
        
        # 将图像转移到GPU
        image = image.to(device)
        
        # 添加噪声
        if self.add_noise:
            image = add_gaussian_noise(image)

        return image

# 加载数据集
dataset = AugmentedDataset(root_dir=r"L:\常惠林\萎凋\自然萎凋\分类\class_3", transform=transform)

# 生成扩增后的图像并保存
output_folder = r"L:\常惠林\萎凋\自然萎凋\分类\AD3" # 设置保存扩增图像的文件夹
os.makedirs(output_folder, exist_ok=True)

for idx in range(len(dataset)):
    image = dataset[idx]
    img_name = f"augmented_with_noise_{idx}.png"
    img_path = os.path.join(output_folder, img_name)
    
    # 将图像转回到CPU以便保存
    image_cpu = image.cpu()
    
    # 转换为PIL图像以便保存
    image_pil = transforms.ToPILImage()(image_cpu)
    image_pil.save(img_path)
    print(f"保存扩增并添加噪声的图像: {img_name}")
print("图像保存完成！")

保存扩增并添加噪声的图像: augmented_with_noise_0.png
保存扩增并添加噪声的图像: augmented_with_noise_1.png
保存扩增并添加噪声的图像: augmented_with_noise_2.png
保存扩增并添加噪声的图像: augmented_with_noise_3.png
保存扩增并添加噪声的图像: augmented_with_noise_4.png
保存扩增并添加噪声的图像: augmented_with_noise_5.png
保存扩增并添加噪声的图像: augmented_with_noise_6.png
保存扩增并添加噪声的图像: augmented_with_noise_7.png
保存扩增并添加噪声的图像: augmented_with_noise_8.png
保存扩增并添加噪声的图像: augmented_with_noise_9.png
保存扩增并添加噪声的图像: augmented_with_noise_10.png
保存扩增并添加噪声的图像: augmented_with_noise_11.png
保存扩增并添加噪声的图像: augmented_with_noise_12.png
保存扩增并添加噪声的图像: augmented_with_noise_13.png
保存扩增并添加噪声的图像: augmented_with_noise_14.png
保存扩增并添加噪声的图像: augmented_with_noise_15.png
保存扩增并添加噪声的图像: augmented_with_noise_16.png
保存扩增并添加噪声的图像: augmented_with_noise_17.png
保存扩增并添加噪声的图像: augmented_with_noise_18.png
保存扩增并添加噪声的图像: augmented_with_noise_19.png
保存扩增并添加噪声的图像: augmented_with_noise_20.png
保存扩增并添加噪声的图像: augmented_with_noise_21.png
保存扩增并添加噪声的图像: augmented_with_noise_22.png
保存扩增并添加噪声的图像: augmented_with_noise_23.png
保存